In [71]:
import json
import os
from os import listdir
from os.path import isfile, join
import librosa
import json
import torch
import IPython.display as ipd
import os
import soundfile as sf
import torchaudio
# Import the data augmentation component from ASR collection
from nemo.collections.asr.parts import perturb, segment

# Resample noise sound files to 16k (if require)

In [72]:
original_files = []

dataset_dir = 'background_noise'
recordings = [ os.path.join(dataset_dir, name) for name in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir, name)) ]

for file in recordings:
    original_files.append(file)

resampled_path =  'background_noise/'

for file in original_files:

    y, sr = torchaudio.load(file)
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel

    resampler = torchaudio.transforms.Resample(sr, 16000)
    y_resampled = resampler(y)
    new_path = os.path.join(resampled_path,file.split("/")[-1])
    
    sf.write(new_path, y_resampled.numpy(), 16000)

In [73]:
original_files

['background_noise/exercise_bike.wav',
 'background_noise/food_court.wav',
 'background_noise/talking_people.wav',
 'background_noise/animals.wav',
 'background_noise/white_noise.wav',
 'background_noise/playground_kids.wav',
 'background_noise/doing_the_dishes.wav',
 'background_noise/fan.wav',
 'background_noise/piano.wav',
 'background_noise/pink_noise.wav',
 'background_noise/crowded.wav',
 'background_noise/running_tap.wav']

# Play sounds

In [74]:
sr = 16000
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

filepath = os.path.join('background_noise/running_tap.wav')
sample_segment = load_audio(filepath)
ipd.Audio(sample_segment.samples, rate=sr)

# Create noise manifest file

In [75]:
resampled_path = "background_noise/"
onlyfiles = [f for f in listdir(resampled_path) if isfile(join(resampled_path, f))]
training_files =[]
for file in onlyfiles:
    training_files.append(file)

manifest_path = 'noise_files/noise_manifest.jsonl'
with open(manifest_path, 'w', encoding="utf-8") as fout:
    for file in training_files:
        duration = librosa.core.get_duration(filename=resampled_path+file)
        metadata = {
            "audio_filepath": resampled_path+file,
            "duration": duration,
            "label": "noise",
            "text": "_",
            "offset": 0.0
        }
        json.dump(metadata, fout, ensure_ascii=False)
        fout.write('\n')